# The Effects of Covid on the Stock Market
#### Analysis by: Don'yelle Allorh-quarcoo, David Ma, Dejan Savic, Mary Stewart, Wendell K
<p>As we are starting to recover from the recent pandemic of Covid-19, there is a ton of history to look back at in just a couple of years. Many topics might be coming to mind such as the shutdown, the explosion of TikTok, facemasks, working from home, and finding new hobbies to learn or do during the very, very gruesome couple months we had to spend alone or with family. One of those hobbies happened to be what our group likes to call, "Stonks!" For our very first project, we were interested to see how much of an effect Covid-19 had on the stock market before, during, and after.
    We decided to use an API module provided by finnhub and data about Covid-19 provided by the World Health Organization and CDC to truly dive deep and code our way into the rabbit hole of "Stonks!" But, first we had to pull our data.</p>

**The Code**
<p>Let's get our hands dirty with finnhub's API module. First, we imported a .csv file with a list of all tickers in the NASDAQ and NYSE exchanges. Then, we used a for loop with our API response to loop through all of the tickers and append Ticker, Company Name, Industry, Country, and Market Cap information to empty lists. From the DataFrame created by the empty lists, we filtered for all US companies with a market cap larger than $10B (Large-Cap). After dropping all duplicate tickers and N/A industries, we randomly sampled each industry for two stocks and created a DataFrame that we were then able to pull in historical stock price data from 2019-2021.</p>
<p>To make things go by even faster for data gathering, we had created functions to call multiple commands with multiple for loops. Instead of brute force coding and assigning a variable to each dataframe, we created a single dictionary for each year, then simply assigned the dataframe as the value to a key, acting as the "variable." We did the same for plotting as well to really speed through and be as efficient as possible in our code.</p> Can take a look at the code below!

In [ ]:
two_stock_df = pd.DataFrame()
for industry in list(clean_company_industry.Industry.unique()):
    df_temp = clean_company_industry.loc[clean_company_industry['Industry']==industry]
    if len(df_temp) > 1:
        random_sample = random.sample(list(df_temp['Ticker']),k=2)
        df_temp = df_temp.loc[df_temp['Ticker'].isin(random_sample)]
        two_stock_df = two_stock_df.append(df_temp)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
two_stock_df
#--------------
ten_industry_df = pd.DataFrame()
ten_industry_list = ['Airlines', 'Automobiles', 'Consumer products', 'Health Care', 'Hotels, Restaurants & Leisure', 'Logistics & Transportation', 'Pharmaceuticals', 'Real Estate', 'Telecommunication', 'Technology']
for industry in ten_industry_list:
    ten_industry_df = ten_industry_df.append(two_stock_df.loc[two_stock_df['Industry']==industry])
        
# Save the DataFrame to a .csv file
ten_industry_df.to_csv('output/ten_industry_df.csv', index=False)

In [2]:
def stock_df(stock_name, beg_year, end_year):
    df_name = pd.DataFrame(finnhub_client.stock_candles(stock_name, 'W', beg_year, end_year)) 
    df_name = df_name.drop(['s'], axis=1)
    rename = {'c':'Close','h': 'High', 'l': 'Low', 'o': 'Open', 't': 'Time', 'v': 'Volume'}
    df_name.rename(columns=rename, inplace=True)
    df_name['Time'] = [datetime.utcfromtimestamp(x).strftime('%Y-%m-%d') for x in df_name['Time']]
    return df_name
#def function to create a stock chart for the price/date and volume
def plot_chart_covid(dataframe, stock_name, year):
    fig= plt.figure(figsize=(12,10))
    top_plt = plt.subplot2grid((7,4), (0,0), rowspan=3, colspan=4)
    top_plt.plot(dataframe.index, dataframe['Close'], color = '#74D3AE')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Stock Price')
    plt.title(f'{stock_name} Performance Over the Year {year}')
    middle_plt = plt.subplot2grid((7,4),(3,0), rowspan=2, colspan=4)
    middle_plt.bar(dataframe.index, dataframe['Cumulative_cases'], color='#DD9787')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Cumulative Cases by Millions')
    bottom_plt = plt.subplot2grid((7,4),(5,0), rowspan=2, colspan=4)
    bottom_plt.bar(dataframe.index, dataframe['Volume'], color='#8A89C0')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Trading Volume by Millions')
    # plt.tight_layout();
    plt.close(fig) #added after saving/checking figs
    
def plot_chart(dataframe, stock_name, year):
    fig= plt.figure(figsize=(12,10))
    top_plt = plt.subplot2grid((5,4), (0,0), rowspan=3, colspan=4)
    top_plt.plot(dataframe.index, dataframe['Close'],color='#74D3AE')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Stock Price')
    plt.title(f'{stock_name} Performance Over the Year {year}')
    bottom_plt = plt.subplot2grid((5,4),(3,0), rowspan=1, colspan=4)
    bottom_plt.bar(dataframe.index, dataframe['Volume'],color='#8A89C0')
    plt.xticks(rotation=45, fontsize=8)
    plt.xlim([min(dataframe.index),max(dataframe.index)])
    plt.ylabel('Trading Volume by Millions')
    # plt.tight_layout();
    plt.close(fig) #added after saving/checking figs

**Pre-Covid, During Covid, Post-Covid**</p>
**Individual Stock Performance**
<p>We first took a look at each stocks performance over the years of Covid-19. The selected stocks that are depicted in the graph images display various performances throughout the year 2019-2021. One can see that each figure reflects that stocks were consistently volatile in 2020. Most of the stocks like AAPL, ALGN, CLX, CTLT, MSFT, and EL, performed better by the end of the year 2020. DAL, LVS, VZ, and VTRS appear to have a lower stock price at the end of 2020 than at the start of the year.</p>

**Stock Sector Performance**
<p>Why don't we take a step back and take a look at each sectors performance? Using pre-Covid(2019) as a constant, we were be able to find out how each sector was affected and how well each stock recovered. The Logistics and Transportation sector actually performed the best during Covid-19(2020), which is shown in the FDX and UPS stock price graphs. The price of the stocks for UPS and FDX actively inclines during the second half of 2020. Post-Covid(2021), the industrial sector ended up suffering the most. This is reflected with the negative linear regression displayed on the graphs. The example stocks would be VZ, DAL, UAL and FDX.</p>

**Volume**
<p>There are many factors that come into play when we look at each stock's volume. It is shown that during Covid-19, the volume of each stock varied. Some had a surge in volume when Covid-19 first started, but for almost all of the stocks selected in our sample, you can infer that there is a direct correlation between the increase of Covid cases and the volume. As Covid cases start to rise, the volume starts to decrease However, the majority of the stocks started and ended within a close margin in the year 2020.</p>
<p>Unlike 2020, 2021 had an amazing year recovering in terms of volume. In fact, there wasn't any correlation between covid cases and the volume of each stock. The volume of each stock had actually increased since 2019 and is staying at a consistant level of about 400 thousand shares traded per week throughout the year.

**Covid Cases VS Stock Price**
<p>Our group had asked, "Could covid cases impact the prices of stonks?" Indeed it did. The number of covid cases had a huge impact on the stock market during the year 2020. Luckily for those in the consumer product industry, as Covid cases started to rise, the price of consumer products rose as well. If you take a look at the stocks, EL and CLX during covid(2020), you can see that the companies are actually benefiting from the increase of Covid cases. Following closely with the consumer products industry, almost all the other industries suffered a major drop in price at the beginning of the year, but ended up bouncing right back up around August. On the other hand, we have the airlines industry. It is the obvious that customers will not be flying often trapped in an air tight bacteria mobile, and their stock prices show it.

**Covid Milestones**
<p>Looking back at all the Covid-19 milestones, we can see that when they announced Covid-19 officially a pandemic, all the stocks had a dramatic drop in price. Following the announcement, states issuing shutdowns also impacted the price and volume of stocks during the year of 2020. As mentioned before, the airlines industry suffered the biggest drop and a slow increase back up to 2019 stock prices. Shortly after, when the Trump Administration signed the Covid-19 relief, we can start to see an increase in stock prices.

**Linear Regression & Future Predictions**
<p>Although using a linear regression is great way to create predictions, it doesn't help too much when we're talking about the stock market. If we take a look at each sectors performance, they all differed. For instance, the travel sector displayed a negative correlation of the stock prices during Covid-19(2020) while the technology and healthcare industries were thriving. 

**Correlation Between Covid and Stocks**
<p>There is indeed a correlation between stock price performance and Covid cases. After reviewing the stock performance in 2020, it is evident that several sectors were positively impacted by Covid. Sectors such as technology, consumer products, and logistics and transportation prices increased. They also continue to increase in the following year of 2021 as well as the others that might not have performed as well. There was a correlation between each stocks volume and Covid during 2020, but completely switched towards the end around December and in 2021. There was not a correlation between the two in 2021 though, as almost all sectors of the market begin to recover.

**Biggest Sector Losers**
<p>A good handful of sectors did end up suffering during the pandemic, but the biggest loss was definitely in the airlines industry. If we take a closer look at the two stocks, DAL and UAL, they dipped just as hard as the others but weren't able to recover as quickly. Some other stocks that couldn't recover as quickly are VZ and FDX where they had a much later correction in stock prices, but were still able to out perform the airlines industry in the end.

**Biggest Sector Winner**
<p>Many of the sectors performed greatly towards the end, but the technology sector definitely came out on top. Consumer products sector had a great increase during August, but had a slower increase than than the technology sector. A close second would definitely be the automobiles industry. Both GM and TSLA were able to make a quick bounce back up after the huge dip, but GM didn't have as steep of a recovery when compared to AAPL and MSFT.